`1. Recommendation system`
`Implementing a recommendation system is critical for businesses and digital platforms that want to thrive in today's competitive environment. These systems use data-driven personalization to tailor content, products, and services to individual user preferences. The latter improves user engagement, satisfaction, retention, and revenue through increased sales and cross-selling opportunities. In this section, you will attempt to implement a recommendation system by identifying similar users' preferences and recommending movies they watch to the study user.`

`To be more specific, you will implement your version of the LSH algorithm, which will take as input the user's preferred genre of movies, find the most similar users to this user, and recommend the most watched movies by those who are more similar to the user.`

`Data: The data you will be working with can be found here.`

`Looking at the data, you can see that there is data available for each user for the movies the user clicked on. Gather the title and genre of the maximum top 10 movies that each user clicked on regarding the number of clicks.`

`1.2 Minhash Signatures`
`Using the movie genre and user_ids, try to implement your min-hash signatures so that users with similar interests in a genre appear in the same bucket.`

`Important note: You must write your minhash function from scratch. You are not permitted to use any already implemented hash functions. Read the class materials and, if necessary, conduct an internet search. The description of hash functions in the book may be helpful as a reference.`

`1.3 Locality-Sensitive Hashing (LSH)`
`Now that your buckets are ready, it's time to ask a few queries. We will provide you with some user_ids and ask you to recommend at most five movies to the user to watch based on the movies clicked by similar users.`

`To recommend at most five movies given a user_id, use the following procedure:`

`Identify the two most similar users to this user.`
`If these two users have any movies in common, recommend those movies based on the total number of clicks by these users.`
`If there are no more common movies, try to propose the most clicked movies by the most similar user first, followed by the other user.`
`Note: At the end of the process, we expect to see at most five movies recommended to the user.`

`Example: assume you've identified user A and B as the most similar users to a single user, and we have the following records on these users:`

`User A with 80% similarity`

`User B with 50% similarity`

`user	movie title	#clicks`

`A	Wild Child	20`

`A	Innocence	10`

`A	Coin Heist	2`

`B	Innocence	30`

`B	Coin Heist	15`

`B	Before I Fall	30`

`B	Beyond Skyline	8`

`B	The Amazing Spider-Man	5`

`Recommended movies in order:`

`Innocence`

`Coin Heist`

`Wild Child`

`Before I Fall`

`Beyond Skyline`


Gather the title and genre of the maximum top 10 movies that each user clicked on regarding the number of clicks:

In [1]:
import pandas as pd

df = pd.read_csv('netflix.csv')

def bestmovies(user):
    #The dictionary for all users
    users_dict = df.groupby('user_id').apply(lambda x: dict(x['movie_id'].value_counts())).to_dict()
    #The dictionary for the specified user
    topmovies = users_dict.get(user, {})
    # Sorting the dictionary by the value in descending order
    sorted_topmovies = dict(sorted(topmovies.items(), key=lambda item: item[1], reverse=True))
    # Return only the first 10 key-value pairs
    return dict(list(sorted_topmovies.items())[:10])

def moviegenres(user):
    # Creating a dictionary with movie_id as key and genre as value
    movie_genre_dict = pd.Series(df.genres.values,index=df.movie_id).to_dict()

    top_10_movies = bestmovies(user)

    # Create a dictionary with the top 10 movies as keys and their genres as values
    moviegenre = {movie: movie_genre_dict[movie] for movie in top_10_movies.keys()}
    return moviegenre

`1.2 Minhash Signatures`
`Using the movie genre and user_ids, try to implement your min-hash signatures so that users with similar interests in a genre appear in the same bucket.`

Making list of genres:

In [2]:

# Splitting the genres and expanding them into separate rows
s = df['genres'].str.split(',').apply(pd.Series, 1).stack()

# Removing whitespaces, dropping duplicates and sorting
s.index = s.index.droplevel(-1)
s.name = 'genres'
del df['genres']
df = df.join(s)
genres_list = df['genres'].str.strip().drop_duplicates().sort_values().tolist()

print(genres_list)


KeyboardInterrupt: 

Making whichgenres column:

In [3]:
# Creating a new column with a list of 27 zeros
df['whichgenres'] = [[0]*27 for _ in range(len(df))]

# Updating the new column based on the presence of genres
for i, genre in enumerate(genres_list):
    df.loc[df['genres'].str.contains(genre), 'whichgenres'] = df.loc[df['genres'].str.contains(genre), 'whichgenres'].apply(lambda x: x[:i] + [1] + x[i+1:])

# Saving the updated dataframe to a new csv file
df.to_csv('Netflix_updated.csv', index=False)

Hash functions:

In [4]:
def h1(k):
    return (5*k + 3)%97
def h2(k):
    return(10*k + 3)%97
def h3(k):
    return(15*k + 3)%97
def h4(k):
    return(20*k + 3)%97
def h5(k):
    return(25*k + 3)%97
def h6(k):
    return(30*k + 3)%97
def h7(k):
    return(35*k + 3)%97
def h8(k):
    return(40*k + 3)%97
def h9(k):
    return(45*k + 3)%97
def h10(k):
    return(50*k + 3)%97
def h11(k):
    return(55*k + 3)%97
def h12(k):
    return(60*k + 3)%97

Making additional column with results of hash functions:

In [5]:
df = pd.read_csv('Netflix_updated.csv')
# Apply the hash functions to each row number
df['hash_results'] = df.index.to_series().apply(lambda x: [h1(x), h2(x), h3(x), h4(x), h5(x), h6(x), h7(x), h8(x), h9(x), h10(x), h11(x), h12(x)])

# Overwrite the original CSV file with the updated DataFrame
df.to_csv('Netflix_updated.csv', index=False)

In [6]:
# Define your minh function
def minh(elements):
    return min(h6(x) for x in elements)

# Apply the minh function to each subset of 4 elements in the hash_results column
df['minh_results'] = df['hash_results'].apply(lambda x: [minh(x[i:i+4]) for i in range(0, len(x), 4)])

# Overwrite the original CSV file with the updated DataFrame
df.to_csv('Netflix_updated.csv', index=False)
print(df.head())

   Unnamed: 0             datetime  duration  \
0       58773  2017-01-01 01:15:09       0.0   
1       58773  2017-01-01 01:15:09       0.0   
2       58773  2017-01-01 01:15:09       0.0   
3       58774  2017-01-01 13:56:02       0.0   
4       58774  2017-01-01 13:56:02       0.0   

                                title release_date    movie_id     user_id  \
0  Angus, Thongs and Perfect Snogging   2008-07-25  26bd5987e8  1dea19f6fe   
1  Angus, Thongs and Perfect Snogging   2008-07-25  26bd5987e8  1dea19f6fe   
2  Angus, Thongs and Perfect Snogging   2008-07-25  26bd5987e8  1dea19f6fe   
3        The Curse of Sleeping Beauty   2016-06-02  f26ed2675e  544dcbc510   
4        The Curse of Sleeping Beauty   2016-06-02  f26ed2675e  544dcbc510   

     genres                                        whichgenres  \
0    Comedy  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1     Drama  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...   
2   Romance  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

A function that takes a user_id as input and returns user ids of the 2 users that are most similar to the user from the input:

In [2]:


# Create dictionaries to map user_ids and movie_ids to their minh_results
user_minh_dict = df.set_index('user_id')['minh_results'].to_dict()
movie_minh_dict = df.set_index('movie_id')['minh_results'].to_dict()

def users_minh(user_id):
    return user_minh_dict.get(user_id, [])

def movies_minh(movie_id):
    return movie_minh_dict.get(movie_id, [])

def points(list1, list2):
    # Flatten the lists
    flat_list1 = [item for sublist in list1 for item in sublist]
    flat_list2 = [item for sublist in list2 for item in sublist]
    
    # Compare the flattened lists
    count = sum(a == b for a, b in zip(flat_list1, flat_list2))
    return count

def movies_concerned(user_id):
    user_minh_results = users_minh(user_id)
    concerned_movie_ids = []
    
    for movie_id, movie_minh_results in movie_minh_dict.items():
        if points(user_minh_results, movie_minh_results) > 0:
            concerned_movie_ids.append(movie_id)
    
    return concerned_movie_ids


NameError: name 'df' is not defined

In [4]:
user_id = "b15926c011"
print(movies_concerned(user_id))

NameError: name 'movies_concerned' is not defined